# PDFパーサーの使い方 

以下のサイトを参考にしました．  
[【PDFMiner】PDFからテキストの抽出](https://qiita.com/mczkzk/items/894110558fb890c930b5)  
[Programming with PDFMine](https://www.unixuser.org/~euske/python/pdfminer/programming.html)

In [1]:
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTContainer, LTTextBox
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [2]:
from pathlib import Path

## LayoutオブジェクトからLTTextBoxのリストを取得する関数

`device`オブジェクトの`get_result`メソッドが返す`layout`オブジェクトのうち，`LTTextBox`(テキストが入っている)のみ取り出し，リストにする．

In [3]:
def find_textboxes_recursively(layout):
    """
    再帰的にテキストボックス（LTTextBox）を探して、テキストボックスのリストを取得する。
    """
    # LTTextBoxを継承するオブジェクトの場合は1要素のリストを返す。
    if isinstance(layout, LTTextBox):
        text_boxes = [layout]
        return text_boxes  # 返すのはリスト

    # LTContainerを継承するオブジェクトは子要素を含むので、再帰的に探す。
    if isinstance(layout, LTContainer):
        text_boxes = []
        for child in layout:
            text_boxes.extend(find_textboxes_recursively(child))  # 再帰的にリストをextend
            
        return text_boxes

    return []  # 何も取得できなかった場合は空リストを返す

## パースに必要なクラスの作成 

In [4]:
# Layout Analysisのパラメーターを設定。縦書きの検出を有効にする。
laparams = LAParams(detect_vertical=True)

# 共有のリソースを管理するリソースマネージャーを作成。
resource_manager = PDFResourceManager(caching=False)  # この引数によってエラーが出なくなる

# ページを集めるPageAggregatorオブジェクトを作成。
device = PDFPageAggregator(resource_manager, laparams=laparams)

# Interpreterオブジェクトを作成。
interpreter = PDFPageInterpreter(resource_manager, device)

## 二段組専用のソート方法 

In [5]:
class Sort2Column():
    def __init__(self, layout_x0, layout_x1):
        self.half_x = (layout_x0 + layout_x1)/2
    
    def __call__(self, text_box):
        if text_box.x0 < self.half_x:
            left_or_right = -1  # it mean left
            
        else:
            left_or_right = 1  # it mean right
            
        return (left_or_right, -text_box.y1)

In [6]:
sorted([0,2,1])

[0, 1, 2]

## ファイルを読み込み，パースを行う

qiitaの記事では，documentオブジェクトは必要ない`PDFPage.get_pages`メソッドを利用する．このメソッドはファイルオブジェクトを引数にとる．

In [9]:
file_path = Path("./PDFs/IS1-20.pdf")

In [10]:
with open(file_path, "rb") as f:
    for page in PDFPage.get_pages(f):
        interpreter.process_page(page)  # ページを処理する。
        layout = device.get_result()  # LTPageオブジェクトを取得。
        text_boxes = find_textboxes_recursively(layout)      

        # text_boxの座標値毎にソート，複数キーのソート
        #text_boxes.sort(key=lambda text_box: (-text_box.y1, text_box.x0))  # y1がy座標，x0がx座標らしい(画像座標なので，y1を負に)
        #text_boxes.sort(key=lambda text_box: (text_box.x0, -text_box.y1)) # 正直二段組のデータでの取得は難しく，ある程度1段踏み前提
        
        # 二段組のソート
        sort2column = Sort2Column(layout_x0=layout.x0, layout_x1=layout.x1)
        text_boxes.sort(key=sort2column)
        for box in text_boxes:
            print("------------------")
            print(box.get_text().strip())  # 末尾の文字を削除
            print("x0:{},x1:{}".format(box.x0, box.x1))
            print("y0:{},y1:{}".format(box.y0, box.y1))
            
        print("-------pages---------")

------------------
IS1-20
SO1-20
x0:15.0,x1:69.36
y0:810.2,y1:843.32
------------------
第25回画像センシングシンポジウム
x0:297.5,x1:435.693
y0:825.98,y1:836.248
------------------

x0:67.224,x1:70.15968000000001
y0:786.9408,y1:796.8144
------------------
画像の局所的な鮮明度合いを用いた照明変動にロバストな物体認識技術
x0:112.58,x1:485.62
y0:740.908,y1:753.808
------------------
矢野  泰樹†  木村  宣隆†  
†株式会社  日立製作所
x0:230.81,x1:379.39
y0:692.908,y1:723.808
------------------
E-mail: taiki.yano.xt@hitachi.com
x0:214.37,x1:383.83
y0:674.908,y1:685.816
------------------
Abstract
x0:151.46,x1:195.17
y0:643.948,y1:654.856
------------------
製造・物流分野での労働力安定供給に向け，人の
x0:67.224,x1:287.12544
y0:621.65104,y1:632.21104
------------------
物理的な非定型作業を代替できるロボットの開発が
x0:56.64,x1:287.03144
y0:605.5710399999999,y1:616.13104
------------------
求められている．さまざまな作業環境での代替実現
x0:56.64,x1:286.96304
y0:589.58104,y1:600.1410400000001
------------------
には，作業環境の照明状況によらずに種々の物体を
認識できる必要がある．そこで，本研究では，画像
の局所的な鮮明度が照明条件によって変わること
に着目し，局所鮮明度に応じて対象物の類似度
評価方法を適切に自動選択する認識アーキテク
チャを

------------------
IS1-20
SO1-20
x0:15.0,x1:69.36
y0:810.2,y1:843.32
------------------
第25回画像センシングシンポジウム
x0:297.5,x1:435.693
y0:825.98,y1:836.248
------------------

x0:67.224,x1:70.15968000000001
y0:786.9408,y1:796.8144
------------------
図  2  提案手法の概要
x0:126.14,x1:230.98568
y0:567.0508,y1:578.18104
------------------

x0:67.224,x1:70.15968000000001
y0:550.7307999999999,y1:560.6043999999999
------------------
る方法を提案する  (図  2)．これにより，照明変動が
生じた物品に対しても，誤認識を増加させることなく
x0:56.64,x1:287.00048
y0:519.62104,y1:546.1410400000001
------------------
正しく認識することが可能になる．以下に，その詳細
な処理方法を記述する．
x0:56.64,x1:286.76352000000014
y0:487.0108,y1:514.10104
------------------
A)  フーリエ変換を利用した局所鮮明度の算出 
まず始めに局所的な模様の鮮明度合いを定量化
x0:67.224,x1:286.86143999999996
y0:455.54103999999995,y1:482.18104
------------------
する必要がある．本研究では，画像の局所領域に対
x0:56.64,x1:287.1044
y0:439.58104,y1:450.14104
------------------
して離散フーリエ変換を実行して得られた各周波数
x0:56.64,x1:287.06311999999997
y0:423.60103999999995,y1:434.16103999999996
------------------
の振幅

------------------
IS1-20
SO1-20
x0:15.0,x1:69.36
y0:810.2,y1:843.32
------------------
第25回画像センシングシンポジウム
x0:297.5,x1:435.693
y0:825.98,y1:836.248
------------------

x0:67.224,x1:70.15968000000001
y0:786.9408,y1:796.8144
------------------
から，鮮明度に応じて再評価する領域を適切に選
択する必要があると考えられる．
x0:56.64,x1:287.18592
y0:727.0408,y1:754.13104
------------------

x0:56.64,x1:59.57568
y0:710.8408,y1:720.7144
------------------
6  まとめ
x0:58.2,x1:127.22
y0:685.828,y1:698.728
------------------
本研究では，種々の照明条件のもとでの対象物品認
x0:56.64,x1:287.12255999999996
y0:663.65104,y1:674.21104
------------------
識を可能とするために，画像の局所的な鮮明度合に
x0:56.64,x1:286.77408
y0:647.5710399999999,y1:658.13104
------------------
応じて模様の類似度評価に使用する特徴種類を適
x0:56.64,x1:287.21
y0:631.61104,y1:642.1710400000001
------------------
切に選択する認識アーキテクチャを開発した．提案
x0:56.64,x1:287.09000000000003
y0:615.65104,y1:626.21104
------------------
手法では，画像の局所領域ごとに使用する特徴種類
x0:56.64,x1:287.02752000000004
y0:599.5710399999999,y1:610.13104
------------------
を選択するため，影や照明の映り込みによって

### documentオブジェクトを利用する場合 